# Лабораторная работа №6

**Сверточные нейронные сети**

---

**Впишите в эту ячейку ваши ФИО, группу**.

ФИО:

Группа:

---


## Загрузка данных

В данной работе будет использоваться учебный датасет с изображениями персонажей из Симпсонов. Код для скачивания и распаковки приведен ниже, его требуется только выполнить и вo вкладке Files должна появиться папка `data`, а в ней папки `train` и `test`.

In [3]:
%%time

! wget -nc http://labcolor.space/rgb-test.zip
! unzip -o -q rgb-test.zip -d data
! wget -nc http://labcolor.space/rgb-train.zip
! unzip -o -q rgb-train.zip -d data

--2023-11-09 07:39:28--  http://labcolor.space/rgb-test.zip
Resolving labcolor.space (labcolor.space)... 31.31.196.3, 2a00:f940:2:2:1:1:0:25
Connecting to labcolor.space (labcolor.space)|31.31.196.3|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3031569 (2.9M) [application/zip]
Saving to: ‘rgb-test.zip’

rgb-test.zip        100%[===================>]   2.89M  2.90MB/s    in 1.0s    

2023-11-09 07:39:30 (2.90 MB/s) - ‘rgb-test.zip’ saved [3031569/3031569]

--2023-11-09 07:39:30--  http://labcolor.space/rgb-train.zip
Resolving labcolor.space (labcolor.space)... 31.31.196.3, 2a00:f940:2:2:1:1:0:25
Connecting to labcolor.space (labcolor.space)|31.31.196.3|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12154147 (12M) [application/zip]
Saving to: ‘rgb-train.zip’

rgb-train.zip       100%[===================>]  11.59M  8.40MB/s    in 1.4s    

2023-11-09 07:39:32 (8.40 MB/s) - ‘rgb-train.zip’ saved [12154147/12154147]

CPU times: user 60.2 

## Создание объекта Dataset

Так как изображения в датасете организованы по папкам, где имя папки является ярлыком для данных, то мы можем воспользоваться базовым классом `ImageFolder`.

Одним из параметров является `transform`, для которого необходимо скомпоновать преобразования для наших изображений. В pytorch для преобразований сейчас есть два набора функций V1 и V2 и рекомендуется использовать V2, хоть напротив многих функций указано состояние beta.

Для компоновки функции из модуля v2 используйте `Compose`. Вам понадобится обязательно:
* ToImage() - преобразование в `Image` (подкласс torch.Tensor)
* RandomVerticalFlip() - случайное отзеркаливание
* ToDtype(torch.float32, scale=True) - преобразование из int во float
* Normalize() - нормализация изображений по полученным средним и стандартным отклонениям.

По желанию:
* RandomRotation() - поворот на случайный угол в указанном диапазоне
* Можете попробовать и другие варианты преобразований. [Документация API V2](https://pytorch.org/vision/stable/transforms.html#v2-api-reference-recommended)

В качестве примера будет показана работа с созданием Dataset для получения статистик изображения. Вам же необходимо будет создать `transforms` для обучения и проверки. При обучении вы используете весь набор обязательных преобразований, при обучении вам требуется только преобразовать изображение к тензору с плавающей точкой и провести нормализацию.

### Получение статистик для нормализации

In [2]:
import torch
from torchvision.transforms import v2

transforms_stats = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
])

In [ ]:
from torchvision.datasets import ImageFolder

stats_dataset = ImageFolder(root="./data/train", transform=transforms_stats)

In [ ]:
imgs = [item[0] for item in stats_dataset]
imgs = torch.stack(imgs, dim=0).numpy()

mean_r = imgs[:,0,:,:].mean()
mean_g = imgs[:,1,:,:].mean()
mean_b = imgs[:,2,:,:].mean()
print(f"Means R, G, B: {mean_r,mean_g,mean_b}")

std_r = imgs[:,0,:,:].std()
std_g = imgs[:,1,:,:].std()
std_b = imgs[:,2,:,:].std()
print(f"Std R, G, B: {std_r,std_g,std_b}")

**Почему значения средних и стандартных отклонений мы получаем только для обучающей выборки?**

Ваш ответ:


---


Используйте выведенные выше значения средних и стандартных отклонений в качестве аргументов функции `Normalize`.

In [ ]:
transforms_train = v2.Compose([
#
# Ваш код
#
])

transforms_test = v2.Compose([
#
# Ваш код
#
])

Теперь, когда есть необходимые `transforms` можно создать ImageFolder, указав в `root` путь до выборки и `transforms` в `transform`.

In [ ]:
train_dataset = ImageFolder() #  Ваш код

# по аналогии создайте датасет для проверочной части

#
#  Ваш код
#

In [ ]:
# Выведите информацию о количестве изображений в обеих выборках и сколько классов в ваших данных.
# Воспользуйтесь атрибутом classes для получения количества классов.
# В объекте Dataset вы можете получить любой объект по индексу, вам будет возвращен кортеж из тензора и индекса класса.
# shape у тензора поможет ответить на вопросы о каналах и размерах изображений.
#
# Ваш код
#

print(f'Количество изображений в обучающей выборке {}')
print(f'Количество изображений в тестовой выборке {}')
print(f'Количество классов {}')
print(f'Количество каналов в изображении {}')
print(f'Высота и ширина изображения {}')

## Создание DataLoader

Далее необходимо подготовить три загрузчика данных:

1. Обучающий
2. Проверочный
3. Тестовый

Тестовый загрузчик делается из тестового Dataset, а обучающий и проверочный необходимо создать, используя [SubsetRandomSampler](https://pytorch.org/docs/stable/data.html#torch.utils.data.SubsetRandomSampler), для его работы требуется массив индексов, по которым в дальнейшем загрузчик будет в случайном порядке брать изображения и лейблы.

In [ ]:
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

val_size = 0.2

train_samples = len(train_dataset)
train_indices = list(range(train_samples))

split_value = int(np.floor(val_size * train_samples))
np.random.shuffle(train_indices)

train_idx, val_idx = train_indices[split_value:], train_indices[:split_value]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

**Опишите своими словами, что делает каждая строчка кода в предыдущей ячейке.**

Ваш ответ:

1.
2.
3.
4. ...

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = # Ваш код

train_loader = DataLoader() # не забудьте использовать нужный sampler

val_loader = DataLoader() # не забудьте использовать нужный sampler

test_loader = DataLoader()

**Какую задачу решает Dataloader?**

Ваш ответ:

**Почему использование трех выборок (обучающей, валидационной, тестовой) считается хорошей практикой?**

Ваш ответ:


## Создание модели

За основу можно взять модель LeNet-5, но скорее всего вам придется ее адаптировать под свою задачу, так как в большинстве случаев она написана под черно-белые изображения размером 32 на 32 пикселя.

Сверточные нейронные сети состоят из двух частей:
1. Слои свертки(функции свертки, активации, субдискретизации)
2. Полносвязные слои (MLP)

Слои можно объединить с помощью `nn.Sequential()`. А класс вашей модели должен наследоваться от `nn.Module`.

`def forward()` определяет прямой ход и должна возвращать итоговый результат работы модели - в данном случае логиты.

In [ ]:
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Два сверточных слоях (nn.Sequential):
        # функция свертки (Conv2d)
        # Функция активации
        # Функция субдескретизации (MaxPool2d)
        self.layer1 = nn.Sequential()
        self.layer2 = nn.Sequential()

        # Полносвязная часть
        # Линейный слой
        # Активация
        # Линейный слой
        # Активация
        # Линейный слой - на выходе количество классов

    def forward(self, x):
        # Из слоев и функций собираете прямой ход модели
        # Не забудьте между сверточными слоями и полносвязными сделать уплощение
        return out

**Опишите суть операции свертки.**

Ваш ответ:

**Опишите суть операции субдискретизации.**

Ваш ответ:



## Создание объекта модели, функции потерь и оптимизатора

В качестве функции потерь будет использована перекрестная энтропия, в задании MLP вы фактически ее реализовали, но через набор отдельных функций.

В качестве оптимизатора можете взять стохастический градиентный спуск или Adam.

In [ ]:
num_classes = # Ваше число
num_epochs = 10   # Пробуйте разные значения
learning_rate = 0.005  # Пробуйте разные значения

model = ConvNet(num_classes)

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(
    # Ваш код
)

## Цикл обучения

Попробуйте разные гиперпараметры для вашей модели. Попробуйте улучшить результат от первой попытки.

Вы можете по своему желанию добавить графики потерь и точности от эпохи.

In [ ]:
%%time

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        model.train()

        # Forward pass
        # Прямой ход:
        # Расчет вывода модели
        # Расчет потерь

        # Обнуление градиентов
        # Обратный ход
        # Шаг оптимизатора

    print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    # Валидация
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in ___:
            # Расчет вывода модели
            # С помощью torch.max можно получить индекс класса
            # К total прибавляем количество данных в этом батче
            # К correct прибавляем сумму совпадений между предсказанными классами и истинными

        print(f'Точность (accuracy) {100 * correct / total}')

## Итоговая оценка

Часть кода, которую вы реализовали для оценки модели на валидационной выборке, можно использовать для финальной проверки, указав нужный loader.

In [ ]:
with torch.no_grad():
    model.eval()
    correct = 0
    total = 0
    for images, labels in ___:
      #
      # Ваш код
      #

    print(f'Точность (accuracy) {100 * correct / total}')

**Точность работы модели на тестовой выборке**

Ваш ответ: